In [7]:
import numpy as np
import cv2
import keras
import glob
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout,  BatchNormalization , Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from matplotlib import pyplot as plt
from PIL import Image
from tf_keras_vis.utils import print_gpus
import tf_keras_vis
import pydot
import graphviz

In [8]:
train_x = np.load('trainx.npy')
train_y = np.load('trainy.npy')
val_x = np.load('valx.npy')
val_y = np.load('valy.npy')

In [9]:
IMG_SIZE = 300
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [ ]:
vgg = tf.keras.models.load_model('corona.h5')
densenet = tf.keras.models.load_model('corona_densenet169.h5')
resnet = tf.keras.models.load_model('corona_resnet.h5')
vgg._name = 'vgg19'
densenet._name = 'densenet'
resnet._name = 'resnet'
vgg.trainable = False
densene.trainable = False
resnet.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)

y1 = vgg(inputs)
y2 = densenet(inputs)
y3 = resnet(inputs)
outputs = tf.keras.layers.Concatenate(axis=-1)([y1, y2, y3])
outputs = tf.keras.layers.Dense(4096, activation='relu')(outputs)
outputs = tf.keras.layers.Dense(3, activation='softmax')(outputs)
ensemble_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
ensemble_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
ensemble_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
vgg19 (Model)                   (None, 3)            16828227    input_2[0][0]                    
__________________________________________________________________________________________________
densenet (Model)                (None, 3)            19475011    input_2[0][0]                    
__________________________________________________________________________________________________
resnet (Model)                  (None, 3)            31992707    input_2[0][0]                    
____________________________________________________________________________________________

In [ ]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
ensemble_model.fit(train_x, train_y, epochs=8, batch_size=64, validation_data=(val_x, val_y), callbacks=callbacks_list)

In [4]:
test_x = np.load('testx.npy')
test_y = np.load('testy.npy')

In [5]:
results = ensemble_model.evaluate(test_x, test_y)

19/19 [==============================] - 204s 11s/step - loss: 0.0982 - accuracy: 0.9801


In [ ]:
ensemble_model.save('corona-ensemble.h5') 